In [ ]:
!pip install tensorflow
!pip install tensorflow-hub
!pip install tensorflow-text

In [ ]:
!pip install youtube-dl
!pip install datetime

In [ ]:
import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_text as text
import os
from datetime import timedelta, datetime, date, time

import youtube_dl
import requests
import re
import uuid
from youtube_transcript_api import YouTubeTranscriptApi
import json

In [ ]:
model_path = r''+os.getcwd()+'\\jerma_model\\'
model = tf.keras.models.load_model(model_path)

In [ ]:
# Testing
model.summary()

In [ ]:
cand = {"[Laughter]":"","[Music]":"","[Applause]":""}
mid_bit = "="
drift = 0.0

In [ ]:
def createTimestampFile_broken(youtubeID):
    '''
    BROKEN
    
    Youtubes, internal closed caption api leaks css data into generated cc data.
    e.g
    https://www.youtube.com/api/timedtext?v=DDg9TGHoPUU&asr_langs=de%2Cen%2Ces%2Cfr%2Cit%2Cja%2Cko%2Cnl%2Cpt%2Cru&caps=asr&exp=xftt%2Cxctw&xoaf=4&hl=en&ip=0.0.0.0&ipbits=0&expire=1629181838&sparams=ip%2Cipbits%2Cexpire%2Cv%2Casr_langs%2Ccaps%2Cexp%2Cxoaf&signature=BED5369F3C1B5C22831C25756CFEEE0D4DE2D421.0626DC01399FEE8B11F63E1452D40BCC14B3B2B5&key=yt8&kind=asr&lang=en&tlang=en&fmt=vtt
    '''
    global mid_bit
    
    name = "timestamp" + mid_bit
    index = 1
    url = "https://www.youtube.com/watch?v={}".format(youtubeID)
    change = mid_bit+ str(uuid.uuid4().hex)
    ydl = youtube_dl.YoutubeDL({'writesubtitles': True, 'allsubtitles': True, 'writeautomaticsub': True})
    res = ydl.extract_info(url, download=False)
    if not os.path.exists(os.getcwd() + "\\temp\\" + name + res.get("id", None) + ".txt"):
        change = ""
    if res['requested_subtitles'] and res['requested_subtitles']['en']:
        print('Grabbing vtt file from ' + res['requested_subtitles']['en']['url'])
        response = requests.get(res['requested_subtitles']['en']['url'], stream=True)
        '''
        Regex:
        \d = digit
        {} = {# of digits}
        \W = any word character
        \s = white spaces
        '''
        timeStampRegex = re.compile(r'\d{2}\W\d{2}\W\d{2}\W\d{3}\s\W{3}\s\d{2}\W\d{2}\W\d{2}\W\d{3}')
        print(response.text)
        new = timeStampRegex.findall(response.text)
#         f1 = open(name + res.get("id", None) + change + ".txt", "a+")
#         f1.write("{")
#         f1.close()
        if new is None:
            print("Found instance of none")
        else:
            for i in new:
                i = re.sub(" --> ","\",\"end\":\"",i)
                # line = '\"'+str(index)+"\":"+"{\"start\":" + str(i)+"},\n"
                line = "{\"start\":\"" + str(i)+"\"}\n"
                # print(line)
                f1 = open(os.getcwd() + "\\temp\\" + name + res.get("id", None) + change + ".txt", "a+")
                f1.write(line)
                f1.close()
                index += 1
#         f1 = open(name + res.get("id", None) + change + ".txt", "a+")
#         f1.write("}")
#         f1.close()
        if len(res['subtitles']) > 0:
            print('manual captions')
        else:
            print('automatic_captions')
            
        return name + res.get("id", None) + change + ".txt"
    else:
        print('Youtube Video does not have any english captions')
        return None

In [ ]:
def createTimestampFile(video_id):
    global mid_bit
    name = "timestamp" + mid_bit
    change = mid_bit+ str(uuid.uuid4().hex)
    if not os.path.exists(os.getcwd() + "\\temp\\" + name + video_id + ".txt"):
        change = ""
    file_name = name + video_id + change
    transcript_list = YouTubeTranscriptApi.list_transcripts(video_id)
    try:
        transcript = transcript_list.find_manually_created_transcript(['en'])
    except Exception:
        transcript = transcript_list.find_generated_transcript(['en'])
    tran = transcript.fetch()
    
    with open(os.getcwd() + "\\temp\\" +'{}.txt'.format(file_name),'w+') as file:
        for line in tran:
            this_line_dic = {"start":line.get("start"),"duration":line.get("duration")}
            file.write(str(this_line_dic)+"\n")  
    return file_name + ".txt"

In [ ]:
def createTranscriptFile(video_id):
    global mid_bit
    global cand
    
    name = "trans" + mid_bit
    change = mid_bit+ str(uuid.uuid4().hex)
    if not os.path.exists(os.getcwd() + "\\temp\\" + name + video_id + ".txt"):
        change = ""
    file_name = name + video_id + change

    transcript_list = YouTubeTranscriptApi.list_transcripts(video_id)
    try:
        transcript = transcript_list.find_manually_created_transcript(['en'])
    except Exception:
        transcript = transcript_list.find_generated_transcript(['en'])
    tran = transcript.fetch()
    
    with open(os.getcwd() + "\\temp\\" +'{}.txt'.format(file_name),'w+') as file:
        for line in tran:
            newLine = line.get("text")
            i = "" if not [k for k,v in cand.items() if line.get("text") == k] else [k for k,v in cand.items() if line.get("text") == k][0]
            newLine = line.get("text").replace(i,"")
            newLine = newLine.replace("\n","")
            # print(newLine)
            if newLine.strip():
                file.write(newLine+"\n")
    return file_name + ".txt"

In [ ]:
def createCommandFile(timestamp_file_name,trans_file_name):
    global mid_bit
    global drift
    
    video_id = timestamp_file_name.split(mid_bit)[1]
    name = "command"+ mid_bit
    change = mid_bit+ str(uuid.uuid4().hex)
    if not os.path.exists(os.getcwd() + "\\temp\\" + name + video_id + ".txt"):
        change = ""
    file_name = name + video_id + change
    
    with open(os.getcwd() + "\\temp\\" +timestamp_file_name,"r") as timestamp_file:
        timestamp_allLines = timestamp_file.readlines()
    
    with open(os.getcwd() + "\\temp\\" +trans_file_name,"r") as trans_file:
        trans_allLines = trans_file.readlines()
    
    timestamp_max_line = len(timestamp_allLines)
    trans_max_line = len(trans_allLines)
    #print(timestamp_max_line)
    #print(trans_max_line )
    for line in range(timestamp_max_line):
        time_dic = json.loads(timestamp_allLines[line].replace("'",'"'))         
        if(not trans_allLines[line].strip()):
            continue
            
        text = trans_allLines[line]
        predict = model.predict([text])
        print(predict)
        print(timestamp_allLines[line])
        if predict > 0.2 + drift:
            with open(os.getcwd() + "\\temp\\" +'{}.txt'.format(file_name),'a+') as command_file:
                command_file.write(get_command(youtubeID,time_dic["start"],time_dic["duration"]))   
    
    return file_name + ".txt"

In [ ]:
def get_command(video_id,start_raw,duration_raw):
    # format of start and end
    ## e.g 
    # format needed
    ## 00:13:00.00
    # format style: url duration start
    # duration = datetime.combine(date.min, time.fromisoformat(end)) - datetime.combine(date.min, time.fromisoformat(start))
    duration = str(timedelta(seconds=duration_raw)).split(".")[0]
    start = str(timedelta(seconds=start_raw)).split(".")[0]
    return "ffmpeg -ss {} -i ".format(start)+os.getcwd().replace("\\","/") + "/videos/" + video_id + "/" + video_id + ".mp4"+" -t {} -c copy\n".format(duration)   

In [ ]:
if __name__ == '__main__':
    youtubeID = "DDg9TGHoPUU"
    timestamp_file_name = createTimestampFile(youtubeID)
    trans_file_name = createTranscriptFile(youtubeID)
    command_file_name = createCommandFile(timestamp_file_name,trans_file_name)